In [3]:
import nltk
from nltk.corpus import wordnet as reader
import nltk.corpus.reader.wordnet as wordnet
import sqlite3 as sql

In [5]:
DICTIONARY_PATH = 'data/output/dictionary.sqlite'

In [68]:
def initDictionary():
    import os

    # os.remove(DICTIONARY_PATH)

    with sql.connect(DICTIONARY_PATH) as conn:
        conn.execute("""
            CREATE TABLE word (
                word TEXT UNIQUE NOT NULL
            )
        """)
        conn.execute("""
            CREATE TABLE definition (
                wordId INTEGER,
                definition TEXT,
                pos TEXT,
                FOREIGN KEY(wordId) REFERENCES word(ROWID)
            )
        """)
        conn.execute("""
            CREATE TABLE example (
                example TEXT,
                definitionId INTEGER,
                FOREIGN KEY(definitionId) REFERENCES definition(ROWID)
            )
        """)

In [26]:
# need to do this because method suggestions don't work for `nltk.corpus.wordnet`
wn = wordnet.WordNetCorpusReader(reader.root, reader._omw_reader)

In [ ]:
for i, word in enumerate(list(wn.words())):
    if i % 100 == 0:
        print(f'{i}: {word}')
    with sql.connect(DICTIONARY_PATH) as conn:
        wordCursor = conn.execute("""
            INSERT INTO word (word) VALUES(:word)
        """, {
            'word': word,
        })
        wordId = wordCursor.lastrowid

        for syn in wn.synsets(word):
            defCursor = conn.execute("""
                INSERT INTO definition (wordId, definition, pos)
                VALUES (:wordId, :definition, :pos)
            """, {
                'wordId': wordId,
                'definition': syn.definition(),
                'pos': syn.pos()
            })
            defId = defCursor.lastrowid

            for ex in syn.examples():
                if word not in ex:
                    continue
                conn.execute("""
                    INSERT INTO example (example, definitionId)
                    VALUES (:example, :definitionId)
                """, {
                    'example': ex,
                    'definitionId': defId
                })


In [ ]:
for i, syn in enumerate(wn.synsets('good')):
    print(f'=============== {i} ===============')
    print(syn)
    print(syn.definition())
    print(syn.examples())
    print(syn.pos())
    print(syn.lemmas())
    # for lem in syn.lemmas():
    #     def iprint(text):
    #         print(f'\t{text}')
    #     iprint(lem)
    #     iprint(f'antonyms  : {lem.antonyms()}')
    #     iprint(f'causes    : {lem.causes()}')
    #     iprint(f'der_forms: {lem.derivationally_related_forms()}')
    #     iprint(f'in_usage_domains: {lem.in_usage_domains()}')
    #     iprint(f'attributes: {lem.attributes()}')